In [1]:
import numpy as np

np.seterr(divide='ignore', invalid='ignore')
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import math
import heapq
from scipy import signal
from matplotlib.lines import Line2D
from scipy import ndimage as nd
from matplotlib.patches import Circle
from skimage import measure
from skimage.transform import warp, AffineTransform

In [3]:
# Helper function to map points to images
def mapping_self(template, des, matrix, mode):
    rows = template.shape[0]
    cols = template.shape[1]

    for row in range(rows):
        for col in range(cols):
            src_cor = np.array([col, row, 1])
            des_cor = np.matmul(matrix, src_cor)

            if mode == "affine":
                des_cor_x = int(np.round(des_cor[1]))
                des_cor_y = int(np.round(des_cor[0]))
                des[des_cor_x, des_cor_y] = template[row, col]

            elif mode == "homograph":
                des_cor_x = int(np.round(des_cor[1]/des_cor[2]))
                des_cor_y = int(np.round(des_cor[0]/des_cor[2]))
                des[des_cor_x, des_cor_y] = template[row, col]
    return des

def mapping(img_warp, original_image):
    book_location = np.where(img_warp > 0)
    original_image[book_location] = 0
    return img_warp + original_image

# ******************************* a ******************************* #
# use SIFT to find point matches between each image

def sift_matches(image1, image2, ratio):
    img1 = cv2.imread(image1)  # queryImage
    img2 = cv2.imread(image2)  # trainImage

    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good = []
    show = []
    for m, n in matches:
        if m.distance < ratio * n.distance:
            good.append(m)
            show.append([m])

    # cv2.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(img1, kp1, img2, kp2, show, flags=2, outImg=None)
    cv2.imwrite("Sift_matches.jpg", img3)


    return kp1, kp2, good


# ******************** TESTING ******************* #
sift_matches("ori/tmp.png", "t2/masked_car2.png", 0.75)

([<KeyPoint 0x110486960>,
  <KeyPoint 0x1104868d0>,
  <KeyPoint 0x110486c00>,
  <KeyPoint 0x1c2a41f750>,
  <KeyPoint 0x1c2a41f7b0>,
  <KeyPoint 0x1c2a41f720>,
  <KeyPoint 0x1c2a41f630>,
  <KeyPoint 0x1c2a41f5d0>,
  <KeyPoint 0x1c2a41f6c0>,
  <KeyPoint 0x1c2a41f6f0>,
  <KeyPoint 0x1c2a41f660>,
  <KeyPoint 0x1c2a41f810>,
  <KeyPoint 0x1c2a41f870>,
  <KeyPoint 0x1c2a41f600>,
  <KeyPoint 0x1c2a41f780>,
  <KeyPoint 0x1c2a41f7e0>,
  <KeyPoint 0x1c2a41f8a0>,
  <KeyPoint 0x1c2a41f840>,
  <KeyPoint 0x1c2a41f9f0>,
  <KeyPoint 0x1c2a41fa20>,
  <KeyPoint 0x1c2a41fa50>,
  <KeyPoint 0x1c2a41fa80>,
  <KeyPoint 0x1c2a41fab0>,
  <KeyPoint 0x1c2a41fae0>,
  <KeyPoint 0x1c2a41fb10>,
  <KeyPoint 0x1c2a41fb40>,
  <KeyPoint 0x1c2a41fb70>,
  <KeyPoint 0x1c2a41fba0>,
  <KeyPoint 0x1c2a41fbd0>,
  <KeyPoint 0x1c2a41fc00>,
  <KeyPoint 0x1c2a41fc30>,
  <KeyPoint 0x1c2a41fc60>,
  <KeyPoint 0x1c2a41fc90>,
  <KeyPoint 0x1c2a41fcc0>,
  <KeyPoint 0x1c2a41fcf0>,
  <KeyPoint 0x1c2a41fd20>,
  <KeyPoint 0x1c2a41fd50>,
  <K